<a href="https://colab.research.google.com/github/Kaavyesh/Diamond_price_prediction-regression/blob/main/Diamond_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(context="notebook", palette="Spectral", style = 'darkgrid' ,font_scale = 1.5, color_codes=False)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LinearRegression
# Regression
from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV, ElasticNet
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/diamonds.csv"

In [3]:
df = pd.read_csv(URL)
df.head()

,s,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   s        53940 non-null  int64  
 1   carat    53940 non-null  float64
 2   cut      53940 non-null  object 
 3   color    53940 non-null  object 
 4   clarity  53940 non-null  object 
 5   depth    53940 non-null  float64
 6   table    53940 non-null  float64
 7   price    53940 non-null  int64  
 8   x        53940 non-null  float64
 9   y        53940 non-null  float64
 10  z        53940 non-null  float64
dtypes: float64(6), int64(2), object(3)
memory usage: 4.5+ MB


In [5]:
df.describe()

,s,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,26970.500000,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,15571.281097,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,1.000000,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,13485.750000,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,26970.500000,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,40455.250000,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,53940.000000,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


Feature Description :
price price in US dollars ( 326−−18,823)

carat weight of the diamond (0.2--5.01)

cut quality of the cut (Fair, Good, Very Good, Premium, Ideal)

color diamond colour, from J (worst) to D (best)

clarity a measurement of how clear the diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))

x length in mm (0--10.74)

y width in mm (0--58.9)

z depth in mm (0--31.8)

depth total depth percentage = z / mean(x, y) = 2 * z / (x + y) (43--79)

table width of top of diamond relative to widest point (43--95)

In [6]:
df.shape

(53940, 11)

the dataset has 53940 rows and 11 columns

In [7]:
df.columns

Index(['s', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x',
       'y', 'z'],
      dtype='object')

In [8]:
df = df.drop(['s'],axis = 1)

 since the size of the diamond has a high positive correlation with its carat weight, dropping the features of length, width, and depth can help prevent multicollinearity and improve the stability and interpretability of the model.

In [9]:
df = df.drop(['x','y','z'],axis = 1)

In [10]:
# lets make a copy of the data frame
df1 = df.copy()

In [11]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
dtypes: float64(3), int64(1), object(3)
memory usage: 2.9+ MB


In [12]:
df1 = pd.get_dummies(df1,drop_first= True)

In [13]:
df1.head()

,carat,depth,table,price,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_E,color_F,...,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0.23,61.5,55.0,326,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,0.21,59.8,61.0,326,0,0,1,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,0.23,56.9,65.0,327,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,0.29,62.4,58.0,334,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,0.31,63.3,58.0,335,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0


In [14]:
co_relation = df1.corr()
co_relation_target = co_relation['price'].sort_values(ascending = False)
print(co_relation_target)

price            1.000000
carat            0.921591
clarity_SI2      0.128420
table            0.127134
color_I          0.097125
cut_Premium      0.095706
color_J          0.081710
color_H          0.059223
clarity_SI1      0.008957
color_G          0.008556
cut_Very Good    0.006593
cut_Good        -0.000312
clarity_VS2     -0.001062
clarity_VS1     -0.009886
depth           -0.010647
color_F         -0.024161
clarity_IF      -0.049596
clarity_VVS2    -0.052381
clarity_VVS1    -0.095266
cut_Ideal       -0.097175
color_E         -0.101089
Name: price, dtype: float64


In [15]:
import plotly.express as px
px.imshow(co_relation,color_continuous_scale = 'Viridis')

In [16]:
df1 = df1.drop(['depth','table'],axis = 1)

In [17]:
df1.columns

Index(['carat', 'price', 'cut_Good', 'cut_Ideal', 'cut_Premium',
       'cut_Very Good', 'color_E', 'color_F', 'color_G', 'color_H', 'color_I',
       'color_J', 'clarity_IF', 'clarity_SI1', 'clarity_SI2', 'clarity_VS1',
       'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2'],
      dtype='object')

In [18]:
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split

x = df1.drop(['price'],axis = 1)
y = df1['price']

In [19]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 0)
model = IsolationForest(random_state = 0,contamination = 0.01)

In [20]:
outliers = model.fit_predict(x_train)
non_outlier_mask = outliers != -1
x_train , y_train = x_train.loc[non_outlier_mask],y_train.loc[non_outlier_mask]

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning:

X does not have valid feature names, but IsolationForest was fitted with feature names



In [21]:
x_train.shape , y_train.shape

((42721, 18), (42721,))

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Define the models
models = [('Linear Regression', LinearRegression()),
          ('Lasso Regression', Lasso()),
          ('AdaBoost Regression', AdaBoostRegressor()),
          ('Ridge Regression', Ridge()),
          ('Gradient Boosting Regression', GradientBoostingRegressor()),
          ('Random Forest Regression', RandomForestRegressor()),
          ('KNeighbours Regression', KNeighborsRegressor())]

# Create the pipeline for each model
pipelines = []
for name, model in models:
    pipelines.append((name, Pipeline([('model', model)])))

# Fit and evaluate each model
for name, pipeline in pipelines:
    pipeline.fit(x_train, y_train)
    y_pred = pipeline.predict(x_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"{name}:")
    print(f"MAE: {mae:.2f}")
    print(f"MSE: {mse:.2f}")
    print(f"R-squared: {r2:.2f}")
    print("\n")

Linear Regression:
MAE: 800.08
MSE: 1319100.36
R-squared: 0.92


Lasso Regression:
MAE: 795.05
MSE: 1325004.77
R-squared: 0.92


AdaBoost Regression:
MAE: 1053.68
MSE: 2385905.36
R-squared: 0.85


Ridge Regression:
MAE: 799.54
MSE: 1319195.66
R-squared: 0.92


Gradient Boosting Regression:
MAE: 441.21
MSE: 737563.70
R-squared: 0.95


Random Forest Regression:
MAE: 310.28
MSE: 400722.13
R-squared: 0.97


KNeighbours Regression:
MAE: 346.96
MSE: 596419.38
R-squared: 0.96




we can see that the Random Forest Regression model has the lowest MAE and MSE scores and the highest R-squared score.